<a href="https://colab.research.google.com/github/cincysam6/CMSAC_2024/blob/main/Full_Completion_Probability_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyarrow scikit-learn xgboost pandas matplotlib seaborn

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import optuna
import re as re
warnings.filterwarnings('ignore')

In [ ]:
# Read the parquet file
completion_df = pd.read_parquet('/content/drive/MyDrive/CMSAC_2024/full_comp_model_df.parquet')
completion_df['is_complete'] = 1-completion_df['incomplete_pass']

In [ ]:
completion_df = completion_df[completion_df['is_target']==1]

In [ ]:
completion_df.shape


(246919, 72)

In [ ]:
### Set Display Options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [ ]:
print(completion_df.columns.tolist())

['gameId', 'playId', 'nflId', 'frameId', 'x', 'y', 'dir', 's', 'o', 'a', 'coverage_x_1', 'coverage_x_2', 'coverage_x_3', 'coverage_y_1', 'coverage_y_2', 'coverage_y_3', 'coverage_s_1', 'coverage_s_2', 'coverage_s_3', 'coverage_a_1', 'coverage_a_2', 'coverage_a_3', 'coverage_o_1', 'coverage_o_2', 'coverage_o_3', 'coverage_dir_1', 'coverage_dir_2', 'coverage_dir_3', 'rec_cov_dist_1', 'rec_cov_dist_2', 'rec_cov_dist_3', 'qb_x', 'qb_y', 'qb_dir', 'qb_s', 'qb_o', 'qb_a', 'is_target', 'down', 'yardsToGo', 'defendersInBox', 'week', 'dist_from_los', 'dist_from_qb', 'dist_from_ball', 'dist_from_side', 'pff_passCoverageTypeMan', 'pff_passCoverageTypeOther', 'pff_passCoverageTypeZone', 'game_play_id', 'target_prob', 'target_binary', 'nflId_1', 'nflId_2', 'nflId_3', 'nflId_4', 'nflId_5', 'nflId_6', 'nflId_7', 'nflId_8', 'pressure_prob_1', 'pressure_prob_2', 'pressure_prob_3', 'pressure_prob_4', 'pressure_prob_5', 'pressure_prob_6', 'pressure_prob_7', 'pressure_prob_8', 'event', 'second_since_snap'

In [ ]:
cols_to_remove =[
                 'nflId_1','nflId_2','nflId_3','nflId_4','nflId_5','nflId_6','nflId_7','nflId_8',
                 'is_target',
                 'target_prob',
                 'target_binary',
                 'event',
                 'incomplete_pass'
                 ]

In [ ]:
model_df = completion_df.drop(columns=cols_to_remove)

In [ ]:
model_df.head().T

,4,9,14,19,24
gameId,2021090900,2021090900,2021090900,2021090900,2021090900
playId,97.0,97.0,97.0,97.0,97.0
nflId,44896,44896,44896,44896,44896
frameId,6,7,8,9,10
x,40.16,40.22,40.34,40.5,40.69
y,35.09,34.81,34.55,34.31,34.08
dir,169.4,164.55,155.29,145.51,138.03
s,2.98,2.88,2.77,2.83,2.87
o,120.74,123.25,124.93,123.94,120.73
a,1.93,1.98,2.41,2.72,2.8


In [ ]:
model_df = model_df.dropna()

In [ ]:
model_df['game_play_id'] = model_df['gameId'].astype(str) + "_" + model_df['playId'].astype(str)

In [ ]:
model_df.isnull().sum()

,0
gameId,0
playId,0
nflId,0
frameId,0
x,0
y,0
dir,0
s,0
o,0
a,0


In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
model_df.describe()

,gameId,playId,nflId,frameId,x,y,dir,s,o,a,coverage_x_1,coverage_x_2,coverage_x_3,coverage_y_1,coverage_y_2,coverage_y_3,coverage_s_1,coverage_s_2,coverage_s_3,coverage_a_1,coverage_a_2,coverage_a_3,coverage_o_1,coverage_o_2,coverage_o_3,coverage_dir_1,coverage_dir_2,coverage_dir_3,rec_cov_dist_1,rec_cov_dist_2,rec_cov_dist_3,qb_x,qb_y,qb_dir,qb_s,qb_o,qb_a,down,yardsToGo,defendersInBox,week,dist_from_los,dist_from_qb,dist_from_ball,dist_from_side,pff_passCoverageTypeMan,pff_passCoverageTypeOther,pff_passCoverageTypeZone,pressure_prob_1,pressure_prob_2,pressure_prob_3,pressure_prob_4,pressure_prob_5,pressure_prob_6,pressure_prob_7,pressure_prob_8,second_since_snap,is_complete
count,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00,241083.00
mean,2021098182.87,2125.60,46013.54,24.32,59.26,25.94,105.55,4.29,135.10,2.71,62.62,63.27,64.37,25.93,25.99,25.90,3.00,2.76,2.71,2.22,2.06,1.98,240.49,248.00,249.60,135.50,139.69,137.96,4.63,8.56,11.95,50.05,26.70,230.87,1.84,125.90,1.90,1.95,8.83,6.06,4.49,5.02,14.45,13.55,16.68,0.30,0.05,0.65,0.26,0.13,0.07,0.04,0.01,0.00,0.00,0.00,1.78,0.64
std,5313.22,1214.13,4758.48,13.39,24.07,11.44,65.80,2.49,88.20,1.53,23.67,23.80,23.95,11.30,9.54,8.60,2.15,1.95,1.93,1.50,1.39,1.36,79.25,71.18,69.71,90.91,92.15,91.39,3.12,3.61,4.51,23.56,3.66,83.80,1.59,52.46,1.26,0.85,3.82,1.02,2.33,6.68,6.99,6.78,5.66,0.46,0.22,0.48,0.25,0.14,0.08,0.05,0.03,0.01,0.01,0.00,1.23,0.48
min,2021090900.00,54.00,30842.00,6.00,6.26,0.14,0.00,0.00,0.01,0.00,11.74,9.83,8.69,0.55,-0.08,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.14,0.60,0.81,-0.94,-2.99,0.00,0.00,0.00,0.00,1.00,1.00,2.00,1.00,0.00,0.30,0.04,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2021092000.00,1152.00,42589.00,14.00,39.60,16.68,65.59,2.27,78.85,1.54,43.20,43.70,44.85,16.86,18.14,20.08,1.17,1.11,1.08,1.07,0.99,0.93,217.35,230.97,233.32,69.35,72.06,72.40,2.12,6.13,9.08,30.14,23.85,196.48,0.64,92.50,0.95,1.00,7.00,6.00,2.00,1.22,9.58,8.62,12.65,0.00,0.00,0.00,0.10,0.05,0.03,0.01,0.00,0.00,0.00,0.00,0.80,0.00
50%,2021100400.00,2084.00,45573.00,23.00,56.02,24.86,95.55,4.55,100.38,2.65,59.54,60.20,61.33,25.04,25.78,25.98,2.80,2.58,2.51,2.03,1.88,1.78,262.73,264.75,265.44,117.89,122.27,118.12,3.96,8.29,11.56,46.30,26.93,266.33,1.48,121.29,1.75,2.00,10.00,6.00,4.00,2.87,13.63,12.76,17.12,0.00,0.00,1.00,0.16,0.09,0.06,0.03,0.00,0.00,0.00,0.00,1.70,1.00
75%,2021101800.00,3162.00,48119.00,32.00,77.02,35.62,135.76,6.31,181.68,3.79,80.22,80.89,82.24,35.40,33.53,31.47,4.48,4.12,4.04,3.17,2.93,2.83,286.39,286.68,286.63,176.39,183.51,181.62,6.47,10.66,14.61,67.11,29.72,280.66,2.55,154.53,2.69,3.00,10.00,7.00,6.00,6.01,18.99,17.88,21.11,1.00,0.00,1.00,0.32,0.15,0.10,0.07,0.00,0.00,0.00,0.00,2.50,1.00
max,2021110100.00,5223.00,53960.00,203.00,121.90,53.17,360.00,10.65,360.00,14.23,120.02,119.20,119.67,53.19,53.34,52.29,10.39,9.80,9.92,11.08,10.84,11.29,360.00,360.00,360.00,360.00,360.00,360.00,42.79,42.93,52.72,107.95,52.30,360.00,9.24,360.00,13.15,4.00,39.00,11.00,8.00,48.98,58.78,57.96,26.65,1.00,1.00,1.00,1.00,0.98,0.96,0.77,0.49,0.44,0.17,0.07,10.30,1.00


In [ ]:
# Split the data
test = model_df[model_df['week'] > 6].copy()

train = model_df[model_df['week'] <= 6].copy()


# Columns to remove
cols_to_remove = ["is_complete", "gameId",'playId', "week", "frameId","nflId"]

In [ ]:
# Prepare training and test data
train_data = train.drop(cols_to_remove, axis=1)
train_labels = train['is_complete']

test_data = test.drop(cols_to_remove, axis=1)
test_labels = test['is_complete']

In [ ]:
X_train = train_data.drop(columns=['game_play_id'])
y_train = train_labels

In [ ]:
X_train_cols = X_train.columns.tolist()

In [ ]:
X_train_cols

['x',
 'y',
 'dir',
 's',
 'o',
 'a',
 'coverage_x_1',
 'coverage_x_2',
 'coverage_x_3',
 'coverage_y_1',
 'coverage_y_2',
 'coverage_y_3',
 'coverage_s_1',
 'coverage_s_2',
 'coverage_s_3',
 'coverage_a_1',
 'coverage_a_2',
 'coverage_a_3',
 'coverage_o_1',
 'coverage_o_2',
 'coverage_o_3',
 'coverage_dir_1',
 'coverage_dir_2',
 'coverage_dir_3',
 'rec_cov_dist_1',
 'rec_cov_dist_2',
 'rec_cov_dist_3',
 'qb_x',
 'qb_y',
 'qb_dir',
 'qb_s',
 'qb_o',
 'qb_a',
 'down',
 'yardsToGo',
 'defendersInBox',
 'dist_from_los',
 'dist_from_qb',
 'dist_from_ball',
 'dist_from_side',
 'pff_passCoverageTypeMan',
 'pff_passCoverageTypeOther',
 'pff_passCoverageTypeZone',
 'pressure_prob_1',
 'pressure_prob_2',
 'pressure_prob_3',
 'pressure_prob_4',
 'pressure_prob_5',
 'pressure_prob_6',
 'pressure_prob_7',
 'pressure_prob_8',
 'second_since_snap']

In [ ]:
X_train.shape

(180904, 52)

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import numpy as np

def objective(trial):

    # Define the XGBoost parameters with adjusted ranges for more flexibility
    param = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',

        # Loosened ranges for regularization
        'lambda': trial.suggest_loguniform('lambda', 0.01, 10.0),  # Reduced range for less regularization
        'alpha': trial.suggest_loguniform('alpha', 0.01, 10.0),    # Reduced range for less regularization

        # Loosened learning rate range for more flexible learning rate
        'eta': trial.suggest_loguniform('eta', 0.01, 0.3),  # Increased upper bound for more aggressive learning

        # Loosened regularization parameter gamma for less strict control on complexity
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),  # Reduced minimum value for less strict regularization

        # Adjusted subsample and colsample_bytree to allow more randomness and larger feature spaces
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),  # Increased lower bound to allow full sampling
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),  # Increased lower bound to allow full feature set

        # Loosened constraints on tree depth and child weight for potentially more complex trees
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Increased maximum depth to allow deeper trees
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # Reduced lower bound for more flexibility in leaf node sizes

        'tree_method': "hist",
        'device': "cuda",
        'predictor': 'gpu_predictor'
    }

    # Assuming train_data and train_labels are pre-defined
    X_train = train_data.drop(columns=['game_play_id'])  # Drop the 'game_play_id' column
    y_train = train_labels
    groups = train_data['game_play_id']  # Group by 'game_play_id'

    # 5-Fold Cross Validation grouped by 'game_play_id'
    kf = GroupKFold(n_splits=5)
    auc_list = []

    for train_index, test_index in kf.split(X_train, y_train, groups=groups):
        X_train_k, X_val_k = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_k, y_val_k = y_train.iloc[train_index], y_train.iloc[test_index]

        # Ensure no leakage by performing feature engineering within the CV loop

        # Example: Standardization within each fold (uncomment if needed)
        # scaler = StandardScaler()
        # X_train_k = scaler.fit_transform(X_train_k)
        # X_val_k = scaler.transform(X_val_k)

        dtrain_k = xgb.DMatrix(X_train_k, label=y_train_k)
        dval_k = xgb.DMatrix(X_val_k, label=y_val_k)

        # Train the model with early stopping
        bst = xgb.train(param, dtrain_k, num_boost_round=2500, evals=[(dval_k, "validation")], early_stopping_rounds=100, verbose_eval=50)
        preds = bst.predict(dval_k)

        # Calculate AUC
        auc = roc_auc_score(y_val_k, preds)
        auc_list.append(auc)

    # Average AUC over all folds
    avg_auc = np.mean(auc_list)

    return avg_auc


In [ ]:

# Create a study object and specify the direction of optimization
study = optuna.create_study(direction='maximize')  # We want to maximize the PR AUC

# Optimize the objective function using the study
study.optimize(objective, n_trials=100)  # You can adjust n_trials based on how thorough you want the search to be

# Retrieve the best trial
best_trial = study.best_trial
best_params = best_trial.params

# Output the results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', best_params)
print('Best trial value (PR AUC):', best_trial.value)

[I 2024-08-29 18:46:06,815] A new study created in memory with name: no-name-54a0e3b8-ae15-48bc-8ecf-9b9245466941


[0]	validation-auc:0.62484
[50]	validation-auc:0.65643
[100]	validation-auc:0.65955
[150]	validation-auc:0.66266
[200]	validation-auc:0.66469
[250]	validation-auc:0.66531
[300]	validation-auc:0.66549
[350]	validation-auc:0.66581
[400]	validation-auc:0.66611
[450]	validation-auc:0.66575
[481]	validation-auc:0.66549
[0]	validation-auc:0.64406
[50]	validation-auc:0.66920
[100]	validation-auc:0.67029
[150]	validation-auc:0.67112
[200]	validation-auc:0.67202
[250]	validation-auc:0.67356
[300]	validation-auc:0.67393
[350]	validation-auc:0.67346
[400]	validation-auc:0.67410
[450]	validation-auc:0.67388
[479]	validation-auc:0.67339
[0]	validation-auc:0.64877
[50]	validation-auc:0.67486
[100]	validation-auc:0.67833
[150]	validation-auc:0.68158
[200]	validation-auc:0.68367
[250]	validation-auc:0.68519
[300]	validation-auc:0.68603
[350]	validation-auc:0.68621
[400]	validation-auc:0.68646
[450]	validation-auc:0.68629
[500]	validation-auc:0.68599
[0]	validation-auc:0.59875
[50]	validation-auc:0.631

[I 2024-08-29 18:46:12,950] Trial 0 finished with value: 0.6633421769347484 and parameters: {'lambda': 0.08186525463206278, 'alpha': 0.1748615461123534, 'eta': 0.010295611629259759, 'gamma': 2.312961779096363, 'subsample': 0.6144727653356687, 'colsample_bytree': 0.7561438389571791, 'max_depth': 4, 'min_child_weight': 10}. Best is trial 0 with value: 0.6633421769347484.


[0]	validation-auc:0.62375
[50]	validation-auc:0.65989
[100]	validation-auc:0.66584
[150]	validation-auc:0.66635
[200]	validation-auc:0.66625
[222]	validation-auc:0.66578
[0]	validation-auc:0.62880
[50]	validation-auc:0.66687
[100]	validation-auc:0.67275
[150]	validation-auc:0.67434
[200]	validation-auc:0.67420
[250]	validation-auc:0.67429
[300]	validation-auc:0.67370
[332]	validation-auc:0.67333
[0]	validation-auc:0.62996
[50]	validation-auc:0.67630
[100]	validation-auc:0.68173
[150]	validation-auc:0.68324
[200]	validation-auc:0.68347
[250]	validation-auc:0.68336
[300]	validation-auc:0.68332
[327]	validation-auc:0.68188
[0]	validation-auc:0.61272
[50]	validation-auc:0.63498
[100]	validation-auc:0.63755
[150]	validation-auc:0.63710
[200]	validation-auc:0.63783
[250]	validation-auc:0.63837
[300]	validation-auc:0.63934
[350]	validation-auc:0.63983
[400]	validation-auc:0.63958
[450]	validation-auc:0.63979
[500]	validation-auc:0.64037
[550]	validation-auc:0.64062
[600]	validation-auc:0.640

[I 2024-08-29 18:46:18,214] Trial 1 finished with value: 0.6640397035887918 and parameters: {'lambda': 4.033618376275707, 'alpha': 0.013760805479232266, 'eta': 0.033963407921065815, 'gamma': 3.8646451864392923, 'subsample': 0.9014940267596386, 'colsample_bytree': 0.95040425133883, 'max_depth': 3, 'min_child_weight': 2}. Best is trial 1 with value: 0.6640397035887918.


...

[0]	validation-auc:0.62416
[50]	validation-auc:0.65533
[100]	validation-auc:0.66148
[150]	validation-auc:0.66447
[200]	validation-auc:0.66526
[250]	validation-auc:0.66471
[299]	validation-auc:0.66399
[0]	validation-auc:0.64286
[50]	validation-auc:0.66849
[100]	validation-auc:0.67024
[150]	validation-auc:0.67272
[200]	validation-auc:0.67343
[250]	validation-auc:0.67268
[283]	validation-auc:0.67185
[0]	validation-auc:0.64294
[50]	validation-auc:0.67401
[100]	validation-auc:0.67934
[150]	validation-auc:0.68368
[200]	validation-auc:0.68392
[250]	validation-auc:0.68435
[261]	validation-auc:0.68412
[0]	validation-auc:0.60091
[50]	validation-auc:0.63286
[100]	validation-auc:0.63563
[150]	validation-auc:0.63713
[200]	validation-auc:0.63753
[250]	validation-auc:0.63799
[300]	validation-auc:0.63838
[350]	validation-auc:0.63862
[368]	validation-auc:0.63852
[0]	validation-auc:0.63087
[50]	validation-auc:0.65716
[100]	validation-auc:0.65805
[150]	validation-auc:0.65887
[200]	validation-auc:0.65925


[I 2024-08-29 18:56:12,079] Trial 98 finished with value: 0.6634580482248499 and parameters: {'lambda': 0.043084273054629814, 'alpha': 1.1633165136283863, 'eta': 0.015869051445937803, 'gamma': 0.25178975565132133, 'subsample': 0.6272083101416731, 'colsample_bytree': 0.922830797504497, 'max_depth': 4, 'min_child_weight': 7}. Best is trial 96 with value: 0.665102801411253.


[0]	validation-auc:0.62384
[50]	validation-auc:0.64938
[100]	validation-auc:0.65755
[150]	validation-auc:0.66241
[200]	validation-auc:0.66482
[250]	validation-auc:0.66598
[300]	validation-auc:0.66662
[350]	validation-auc:0.66611
[400]	validation-auc:0.66597
[405]	validation-auc:0.66587
[0]	validation-auc:0.63774
[50]	validation-auc:0.65853
[100]	validation-auc:0.66341
[150]	validation-auc:0.66867
[200]	validation-auc:0.67146
[250]	validation-auc:0.67353
[300]	validation-auc:0.67387
[350]	validation-auc:0.67464
[400]	validation-auc:0.67476
[450]	validation-auc:0.67522
[500]	validation-auc:0.67475
[550]	validation-auc:0.67414
[559]	validation-auc:0.67396
[0]	validation-auc:0.63386
[50]	validation-auc:0.66717
[100]	validation-auc:0.67477
[150]	validation-auc:0.67933
[200]	validation-auc:0.68240
[250]	validation-auc:0.68321
[300]	validation-auc:0.68409
[350]	validation-auc:0.68434
[400]	validation-auc:0.68460
[450]	validation-auc:0.68488
[500]	validation-auc:0.68541
[550]	validation-auc:0.

[I 2024-08-29 18:56:20,310] Trial 99 finished with value: 0.6651704535860582 and parameters: {'lambda': 0.03970110480137013, 'alpha': 0.6772611579470493, 'eta': 0.014200341437255539, 'gamma': 0.44644612349844326, 'subsample': 0.6456703137206167, 'colsample_bytree': 0.9346343000712446, 'max_depth': 3, 'min_child_weight': 7}. Best is trial 99 with value: 0.6651704535860582.


Number of finished trials: 100
Best trial parameters: {'lambda': 0.03970110480137013, 'alpha': 0.6772611579470493, 'eta': 0.014200341437255539, 'gamma': 0.44644612349844326, 'subsample': 0.6456703137206167, 'colsample_bytree': 0.9346343000712446, 'max_depth': 3, 'min_child_weight': 7}
Best trial value (PR AUC): 0.6651704535860582


In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)

best_params.update({
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': "hist",
    'device': "cuda",
    'predictor': 'gpu_predictor'
})

print(best_params)

{'lambda': 0.03970110480137013, 'alpha': 0.6772611579470493, 'eta': 0.014200341437255539, 'gamma': 0.44644612349844326, 'subsample': 0.6456703137206167, 'colsample_bytree': 0.9346343000712446, 'max_depth': 3, 'min_child_weight': 7, 'booster': 'gbtree', 'objective': 'binary:logistic', 'eval_metric': 'auc', 'tree_method': 'hist', 'device': 'cuda', 'predictor': 'gpu_predictor'}


In [ ]:

# Train the model with the best parameters
best_model = xgb.train(best_params, dtrain, num_boost_round=2500)

In [ ]:

# Save the model to a JSON file
best_model.save_model("best_xgb_model_full_completion_prob.json")

# Alternatively, use pickle to save the entire model object
with open("/content/drive/MyDrive/CMSAC_2024/best_xgb_model_full_completion_prob.pkl", "wb") as f:
    pickle.dump(best_model, f)


In [ ]:

X_test = test_data.drop(columns=['game_play_id'])
y_test = test_labels
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# Predictions for train set
train_preds = best_model.predict(dtrain)

# Predictions for test set
test_preds = best_model.predict(dtest)

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate AUC for train and test sets
train_auc = roc_auc_score(y_train, train_preds)
test_auc = roc_auc_score(y_test, test_preds)

print(f'Train AUC: {train_auc:.4f}')
print(f'Test AUC: {test_auc:.4f}')

In [ ]:
# Extract feature importance
importance = best_model.get_score(importance_type='gain')

# Convert the feature importance dictionary to a DataFrame
importance_df = pd.DataFrame(importance.items(), columns=['Feature', 'Importance'])
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('XGBoost Feature Importance')
plt.gca().invert_yaxis()  # To have the most important feature at the top
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Convert predicted probabilities to binary outcomes
train_preds_binary = [1 if x > 0.5 else 0 for x in train_preds]
test_preds_binary = [1 if x > 0.5 else 0 for x in test_preds]

# Accuracy
train_accuracy = accuracy_score(y_train, train_preds_binary)
test_accuracy = accuracy_score(y_test, test_preds_binary)

# Precision, Recall, F1-Score
train_precision = precision_score(y_train, train_preds_binary)
test_precision = precision_score(y_test, test_preds_binary)

train_recall = recall_score(y_train, train_preds_binary)
test_recall = recall_score(y_test, test_preds_binary)

train_f1 = f1_score(y_train, train_preds_binary)
test_f1 = f1_score(y_test, test_preds_binary)

# Confusion Matrix
train_conf_matrix = confusion_matrix(y_train, train_preds_binary)
test_conf_matrix = confusion_matrix(y_test, test_preds_binary)

# Classification Report
test_class_report = classification_report(y_test, test_preds_binary)

# Print all metrics
print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

print(f'Train Precision: {train_precision:.4f}')
print(f'Test Precision: {test_precision:.4f}')

print(f'Train Recall: {train_recall:.4f}')
print(f'Test Recall: {test_recall:.4f}')

print(f'Train F1-Score: {train_f1:.4f}')
print(f'Test F1-Score: {test_f1:.4f}')

print(f'Test Confusion Matrix:\n{test_conf_matrix}')
print(f'Test Classification Report:\n{test_class_report}')


In [ ]:
# If you have binary predictions as well:
train_preds_binary = [1 if x > 0.5 else 0 for x in train_preds]
test_preds_binary = [1 if x > 0.5 else 0 for x in test_preds]

# Convert predictions to DataFrame and make sure they match the index of the original data
train_preds_df = pd.DataFrame({
    'complete_prob': train_preds,
    'complete_binary': train_preds_binary
}, index=train.index)

test_preds_df = pd.DataFrame({
    'complete_prob': test_preds,
    'complete_binary': test_preds_binary
}, index=test.index)

In [ ]:
# Join the predictions back to the original data
train_with_preds = train.join(train_preds_df)
test_with_preds = test.join(test_preds_df)

In [ ]:
full_preds = pd.concat([train_with_preds, test_with_preds])

In [ ]:
full_preds[['is_complete','gameId','playId','frameId','nflId','complete_prob']].head(200)

In [ ]:
full_preds.to_parquet("/content/drive/MyDrive/CMSAC_2024/full_completion_preds.parquet", index=False)

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
full_preds.describe()

In [ ]:
full_preds['complete_pass'].value_counts()